In [2]:
import os, re, time, json
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

c:\Users\MSI Gaming PC\source\repos\plankton_classification\models\resnet\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# locad the dataset
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

In [4]:
def preprocess_image_input(input_images):
    input_images = input_images.astype('float32')
    output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
    return output_ims

In [5]:

# preprocess the images
train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

# convert the labels to one-hot encoding
train_Y = tf.keras.utils.to_categorical(training_labels, num_classes=10)
valid_Y = tf.keras.utils.to_categorical(validation_labels, num_classes=10)

In [6]:
def get_base_model(inputs):

    feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
    return feature_extractor

In [7]:
def get_classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

In [8]:
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    base_model = get_base_model(resize)
    classification_output = get_classifier(base_model)

    return classification_output

In [9]:
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs = inputs, outputs = classification_output)
 
  model.compile(optimizer='Adam', 
                loss='categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model

In [19]:
model = define_compile_model()

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d_2 (UpSampling  (None, 224, 224, 3)      0         
 2D)                                                             
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 1024)              2098

In [20]:
EPOCHS = 10

In [21]:
history = model.fit(train_X, train_Y, epochs=EPOCHS, validation_data = (valid_X, valid_Y), batch_size=64, verbose=1)
model.save('resnet_cifar10.h5')

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'model_2/resnet50/conv3_block2_3_bn/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\MSI Gaming PC\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\MSI Gaming PC\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\MSI Gaming PC\source\repos\plankton_classification\models\resnet\.venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\MSI Gaming PC\source\repos\plankton_classification\models\resnet\.venv\lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "c:\Users\MSI Gaming PC\source\repos\plankton_classification\models\resnet\.venv\lib\site-packages\ipykernel\kernelapp.py", line 724, in start
      self

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()